In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
import pandas as pd

In [3]:
df = pd.read_csv("training_dataset1.csv")

In [5]:
X = df[['Num_escuelas',
 'Num_hospitales',
 'Num_iglesias',
 'Num_zonas_tipicas',
   ]]

y = df["Num_viajes"]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [7]:
linear_model = LinearRegression()
linear_model.fit(X, y)

LinearRegression()

In [8]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np 

yfit_linear = pd.DataFrame(linear_model.predict(X_train), index = X_train.index, columns = ["Num_viajes"]).clip(0.)

ypred_linear = pd.DataFrame(linear_model.predict(X_test), index = X_test.index, columns = ["Num_viajes"]).clip(0.)


print('Linear RMSLE :', np.sqrt(msle(y_test, ypred_linear)))
print('Linear MAE :', mae(y_test, ypred_linear))


Linear RMSLE : 0.693876749693628
Linear MAE : 19388.925108526328


# Evaluación del modelo

Debido a que seguiremos iterando y evolucionando el modelo, las siguientes métricas servirán como benchmarck para saber si la siguiente iteración representa una mejora

Linear RMSLE : 0.693876749693628

Linear MAE : 19388.925108526328